In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings

In [12]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
# Importing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [14]:
%run features_testing.ipynb

In [15]:
# Split the data into explanatory and target variables
X = train.drop("TARGET", axis=1).values
y = train["TARGET"].values
X_test = test.values

In [16]:
#from sklearn.preprocessing import PowerTransformer
#
#pt = PowerTransformer(method='yeo-johnson')
#X_pt = pt.fit_transform(X)
#X_test_pt = pt.transform(X_test)

In [17]:
# Standardization
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)
X_test_std = sc.transform(X_test)

In [18]:
# Split the original data into the training data and the validation data
X_train, X_valid, y_train, y_valid = train_test_split(X_std, y, test_size=0.3, stratify=y, random_state=0)

In [19]:
lgb_params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "learning_rate": 0.03,

    "num_leaves": 16,          
    "max_depth": 4,              
    "min_child_samples": 40,
    "min_child_weight": 1e-3,

    "subsample": 0.8,
    "subsample_freq": 1,
    "colsample_bytree": 0.7,

    "reg_alpha": 0.4,    
    "reg_lambda": 0.6,

    "n_estimators": 2000,
    "early_stopping_rounds": 200,
    "random_state": 42,
    "verbose": -1
}

In [20]:
# this will become important when the data grow
import optuna
import lightgbm as lgb

def objective(trial):

    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",

        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", -1, 12),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),

        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),

        "min_child_samples": trial.suggest_int("min_child_samples", 10, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),

        "n_estimators": trial.suggest_int("n_estimators", 200, 3000),
        "verbose": -1
    }

    model = lgb.LGBMClassifier(**params)

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=0)
        ]
    )

    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)

    return auc



study = optuna.create_study(
    direction="maximize",
    study_name="lgbm_optimization"
)

study.optimize(objective, n_trials=50)

print("\nBest AUC:", study.best_value)
print("Best hyperparameters:\n", study.best_params)


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-25 01:39:11,205] A new study created in memory with name: lgbm_optimization


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[413]	valid_0's auc: 0.736492


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-25 01:39:28,072] Trial 0 finished with value: 0.7364922901207483 and parameters: {'num_leaves': 238, 'max_depth': 0, 'learning_rate': 0.0022026225789126717, 'feature_fraction': 0.7574867242714163, 'bagging_fraction': 0.7633346838980356, 'bagging_freq': 2, 'min_child_samples': 54, 'lambda_l1': 9.314393674822856e-08, 'lambda_l2': 2.7190394968328172e-08, 'n_estimators': 413}. Best is trial 0 with value: 0.7364922901207483.


Training until validation scores don't improve for 100 rounds


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-25 01:39:32,899] Trial 1 finished with value: 0.7450032461762222 and parameters: {'num_leaves': 79, 'max_depth': 7, 'learning_rate': 0.01927977029974666, 'feature_fraction': 0.6338454780170465, 'bagging_fraction': 0.6088939163390285, 'bagging_freq': 4, 'min_child_samples': 91, 'lambda_l1': 0.0003919059768423908, 'lambda_l2': 2.1664408558999384e-06, 'n_estimators': 381}. Best is trial 1 with value: 0.7450032461762222.


...

Early stopping, best iteration is:
[244]	valid_0's auc: 0.744162
Training until validation scores don't improve for 100 rounds


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-25 01:46:34,387] Trial 48 finished with value: 0.7463979551005476 and parameters: {'num_leaves': 82, 'max_depth': 5, 'learning_rate': 0.07500022822432262, 'feature_fraction': 0.8000669775833891, 'bagging_fraction': 0.8402379037558965, 'bagging_freq': 7, 'min_child_samples': 31, 'lambda_l1': 6.398845010008367e-05, 'lambda_l2': 9.788326602571199e-08, 'n_estimators': 1265}. Best is trial 24 with value: 0.7493641259546003.


Early stopping, best iteration is:
[195]	valid_0's auc: 0.746398
Training until validation scores don't improve for 100 rounds


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-25 01:46:42,844] Trial 49 finished with value: 0.7423499996602869 and parameters: {'num_leaves': 143, 'max_depth': 0, 'learning_rate': 0.0313592964327413, 'feature_fraction': 0.8857724801907787, 'bagging_fraction': 0.8739765942957983, 'bagging_freq': 4, 'min_child_samples': 49, 'lambda_l1': 0.0015927009556688623, 'lambda_l2': 8.841955516123053e-05, 'n_estimators': 2923}. Best is trial 24 with value: 0.7493641259546003.


Early stopping, best iteration is:
[199]	valid_0's auc: 0.74235

Best AUC: 0.7493641259546003
Best hyperparameters:
 {'num_leaves': 57, 'max_depth': 4, 'learning_rate': 0.03277842323048595, 'feature_fraction': 0.9569717768433371, 'bagging_fraction': 0.8506767379185446, 'bagging_freq': 1, 'min_child_samples': 48, 'lambda_l1': 0.17636584028657937, 'lambda_l2': 0.025244809309038312, 'n_estimators': 2486}


In [ ]:
# lightGBM cause seem like this is the best for the old model
import lightgbm as lgb
# optuna para {'num_leaves': 57, 'max_depth': 4, 'learning_rate': 0.03277842323048595, 'feature_fraction': 0.9569717768433371, 'bagging_fraction': 0.8506767379185446, 'bagging_freq': 1, 'min_child_samples': 48, 'lambda_l1': 0.17636584028657937, 'lambda_l2': 0.025244809309038312, 'n_estimators': 2486}

lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train, eval_set= [(X_valid, y_valid)])

lgbm_train_pred = lgbm.predict_proba(X_train)[:, 1]
lgbm_valid_pred = lgbm.predict_proba(X_valid)[:, 1]

print(f"Train Score: {roc_auc_score(y_train, lgbm_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, lgbm_valid_pred)}")

Train Score: 0.8122957326179272
Valid Score: 0.7411931027094857


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


### under this is the output part


In [22]:
# Make predictions for the test data
# Change model name if needed
pred = lgbm.predict_proba(X_test_std)[:, 1]

/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [23]:
# Put the prediction into the format of submission
sample_sub['TARGET'] = pred
sample_sub

,SK_ID_CURR,TARGET
0,171202,0.028224
1,171203,0.110668
2,171204,0.121044
3,171205,0.099954
4,171206,0.195951
...,...,...
61495,232697,0.270269
61496,232698,0.057006
61497,232699,0.046150
61498,232700,0.083779


In [24]:
# Create the "output" directory if it doesn't exist
output_dir = Path.cwd() / "output"
os.makedirs(output_dir, exist_ok=True)

# Specify the new output file path
output_file = output_dir / "submission.csv"

# Save the CSV file to the "output" directory
sample_sub.to_csv(output_file, index=False)